# More JOIN operations

![rel](https://sqlzoo.net/w/images/1/10/Movie-er.png)

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app07")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@21af06b6

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@5545b746

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val movie = hiveCxt.table("sqlzoo.movie")
val actor = hiveCxt.table("sqlzoo.actor")
val casting = hiveCxt.table("sqlzoo.casting")

movie: DataFrame = [id: int, title: string ... 4 more fields]
actor: DataFrame = [id: int, name: string]
casting: DataFrame = [movieid: int, actorid: int ... 1 more field]

## 1. 1962 movies

List the films where the **yr** is 1962 [Show **id, title**]

In [5]:
movie.filter(movie("yr")===1962).select("id", "title").showHTML()

id,title
10212,A Kind of Loving
10329,A Symposium on Popular Songs
10347,A Very Private Affair (Vie PrivÃ©e)
10648,An Autumn Afternoon
10868,Atraco a las tres
11006,Barabbas
11053,Battle Beyond the Sun (ÐÐµÐ±Ð¾ Ð·Ð¾Ð²ÐµÑ‚)
11199,Big and Little Wong Tin Bar
11230,Billy Budd
11234,Billy Rose's Jumbo


## 2. When was Citizen Kane released?

Give year of 'Citizen Kane'.

In [6]:
movie.filter(movie("title")==="Citizen Kane").select("yr").showHTML()

yr
1941


## 3. Star Trek movies

List all of the Star Trek movies, include the **id**, **title** and **yr** (all of these movies include the words Star Trek in the title). Order results by year.

In [7]:
(movie.filter(lower(movie("title")).contains("star trek"))
 .select("id", "title", "yr").showHTML())

id,title,yr
17770,Star Trek: First Contact,1996
17771,Star Trek: Insurrection,1998
17772,Star Trek: The Motion Picture,1979
17773,Star Trek,2009
17774,Star Trek Generations,1994
17775,Star Trek II: The Wrath of Khan,1982
17776,Star Trek III: The Search for Spock,1984
17777,Star Trek IV: The Voyage Home,1986
17778,Star Trek Nemesis,2002
17779,Star Trek V: The Final Frontier,1989


## 4. id for actor Glenn Close

What **id** number does the actor 'Glenn Close' have?

In [8]:
actor.filter(actor("name")==="Glenn Close").select("id").showHTML()

id
140


## 5. id for Casablanca

What is the **id** of the film 'Casablanca'

In [9]:
movie.filter(movie("title")==="Casablanca").select("id").showHTML()

id
11768


## 6. Cast list for Casablanca

Obtain the cast list for 'Casablanca'.

> _what is a cast list?_  
> The cast list is the names of the actors who were in the movie.

Use **movieid=11768**, (or whatever value you got from the previous question)

In [10]:
(casting.join(actor, casting("actorid")===actor("id"))
 .filter($"movieid"===11768)
 .select("name")
 .showHTML())

name
Peter Lorre
John Qualen
Madeleine LeBeau
Jack Benny
Dan Seymour
Norma Varden
Ingrid Bergman
Conrad Veidt
Leon Belasco
Humphrey Bogart


## 7. Alien cast list

Obtain the cast list for the film 'Alien'

In [11]:
(movie.join(casting, movie("id")===casting("movieid"), joinType="right")
 .join(actor, col("actorid")===actor("id"), joinType="left")
 .filter($"title"==="Alien")
 .select("name")
 .showHTML())

name
John Hurt
Sigourney Weaver
Yaphet Kotto
Harry Dean Stanton
Ian Holm
Tom Skerritt
Veronica Cartwright


## 8. Harrison Ford movies

List the films in which 'Harrison Ford' has appeared

In [12]:
(movie.join(casting, movie("id")===casting("movieid"), joinType="right")
 .join(actor, col("actorid")===actor("id"), joinType="left")
 .filter($"name"==="Harrison Ford")
 .select("title")
 .showHTML())

title
A Hundred and One Nights
Air Force One
American Graffiti
Apocalypse Now
Clear and Present Danger
Cowboys & Aliens
Crossing Over
Dead Heat on a Merry-Go-Round
Extraordinary Measures
Firewall


## 9. Harrison Ford as a supporting actor

List the films where 'Harrison Ford' has appeared - but not in the starring role. [Note: the ord field of casting gives the position of the actor. If ord=1 then this actor is in the starring role]

In [13]:
(movie.join(casting, movie("id")===casting("movieid"), joinType="right")
 .join(actor, col("actorid")===actor("id"), joinType="left")
 .filter(($"name"==="Harrison Ford") && ($"ord">1))
 .select("title")
 .showHTML())

title
A Hundred and One Nights
American Graffiti
Apocalypse Now
Cowboys & Aliens
Dead Heat on a Merry-Go-Round
Extraordinary Measures
Force 10 From Navarone
Hawthorne of the U.S.A.
Jimmy Hollywood
More American Graffiti


## 10. Lead actors in 1962 movies

List the films together with the leading star for all 1962 films.

In [14]:
(movie.join(casting, movie("id")===casting("movieid"), joinType="right")
 .join(actor, col("actorid")===actor("id"), joinType="left")
 .filter(($"yr"===1962) && ($"ord"===1))
 .select("title", "name")
 .showHTML())

title,name
A Kind of Loving,Alan Bates
A Symposium on Popular Songs,Paul Frees
A Very Private Affair (Vie PrivÃ©e),Brigitte Bardot
An Autumn Afternoon,Chishu Ryu
Atraco a las tres,JosÃ© Luis LÃ³pez VÃ¡zquez
Barabbas,Anthony Quinn
Battle Beyond the Sun (ÐÐµÐ±Ð¾ Ð·Ð¾Ð²ÐµÑ‚),Aleksandr Shvorin
Big and Little Wong Tin Bar,Jackie Chan
Billy Budd,Terence Stamp
Billy Rose's Jumbo,Doris Day


## 11. Busy years for Rock Hudson

Which were the busiest years for 'Rock Hudson', show the year and the number of movies he made each year for any year in which he made more than 2 movies.

In [15]:
(movie.join(casting, movie("id")===casting("movieid"), joinType="right")
 .join(actor, col("actorid")===actor("id"), joinType="left")
 .filter($"name"==="Rock Hudson")
 .select("yr", "title")
 .groupBy("yr")
 .count()
 .filter($"count" > 2)
 .showHTML())

yr,count
1961,3
1953,5


## 12. Lead actor in Julie Andrews movies

List the film title and the leading actor for all of the films 'Julie Andrews' played in.

> _Did you get "Little Miss Marker twice"?_   
> Julie Andrews starred in the 1980 remake of Little Miss Marker and not the original(1934).
>
> Title is not a unique field, create a table of IDs in your subquery

In [16]:
val b = (movie.join(casting, movie("id")===casting("movieid"), joinType="right")
         .join(actor, col("actorid")===actor("id"), joinType="left")
         .filter($"name"==="Julie Andrews")
         .select("movieid")
         .collect()
         .map(_(0))
         .toList)
(movie.join(casting, movie("id")===casting("movieid"), joinType="right")
 .join(actor, col("actorid")===actor("id"), joinType="left")
 .filter(($"movieid".isin(b: _*)) && ($"ord"===1))
 .select("title", "name")
 .showHTML())

title,name
10,Dudley Moore
Darling Lili,Julie Andrews
Despicable Me,Steve Carell
Duet for One,Julie Andrews
Hawaii,Julie Andrews
Little Miss Marker,Walter Matthau
Mary Poppins,Julie Andrews
Relative Values,Julie Andrews
Shrek the Third,Mike Myers
Star!,Julie Andrews


b: List[Any] = List(
  10016,
  12354,
  12497,
  12766,
  13846,
  15145,
  15476,
  16870,
  17445,
  17765,
  18270,
  20136,
  20136,
  20180,
  20181,
  20509,
  20627,
  21023,
  21154,
  21171,
  21483
)

## 13. Actors with 15 leading roles

Obtain a list, in alphabetical order, of actors who've had at least 15 **starring** roles.

In [17]:
(movie.join(casting, movie("id")===casting("movieid"), joinType="right")
 .join(actor, col("actorid")===actor("id"), joinType="left")
 .filter($"ord"===1)
 .groupBy("actorid", "name")
 .count()
 .filter($"count" >= 15)
 .filter(! isnull($"name"))
 .select("name")
 .orderBy("name")
 .showHTML())

name
Adam Sandler
Al Pacino
Anthony Hopkins
Antonio Banderas
Arnold Schwarzenegger
Barbara Stanwyck
Basil Rathbone
Ben Affleck
Bette Davis
Betty Grable


## 14.
List the films released in the year 1978 ordered by the number of actors in the cast, then by title.

In [18]:
(movie.join(casting, movie("id")===casting("movieid"), joinType="right")
 .join(actor, col("actorid")===actor("id"), joinType="left")
 .filter($"yr"===1978)
 .groupBy("title", "movieid")
 .count()
 .select("title", "count")
 .orderBy($"count".desc, $"title".asc)
 .showHTML())

title,count
The Bad News Bears Go to Japan,50
The Swarm,37
Grease,28
American Hot Wax,27
The Boys from Brazil,26
Heaven Can Wait,25
Big Wednesday,21
A Night Full of Rain,19
A Wedding,19
Orchestra Rehearsal,19


## 15.

List all the people who have worked with 'Art Garfunkel'.

In [19]:
val a = (movie.join(casting, movie("id")===casting("movieid"), joinType="right")
         .join(actor, col("actorid")===actor("id"), joinType="left")
         .filter($"name"==="Art Garfunkel")
         .select("movieid"))
(movie.join(casting, movie("id")===casting("movieid"), joinType="right")
 .join(actor, col("actorid")===actor("id"), joinType="left")
 .filter($"name" !== "Art Garfunkel")
 .join(a, Seq("movieid"))
 .select("name")
 .orderBy("name")
 .showHTML())

name
Beverly Johnson
Bill Paxton
Breckin Meyer
Bruce Jay Friedman
Cecilie Thomsen
Cindy Crawford
Donald Trump
Elio Fiorucci
Ellen Albertini Dow
Frederique van der Wal


a: DataFrame = [movieid: int]

In [20]:
spark.stop()